In [1]:
import pandas as pd
import json 

In [2]:
neurips_data = json.load(open("/home/Ravikumar/Developer/similar-research-papers-predictor/dataset/neurips2022.json","r"))

In [9]:
scraped_data = pd.DataFrame.from_dict(neurips_data.values())

In [10]:
scraped_data.head()

,event_id,title,abstract,url
0,53600,Infinite Recommendation Networks: A Data-Centr...,We leverage the Neural Tangent Kernel and its ...,https://nips.cc/Conferences/2022/Schedule?show...
1,55724,GLOBEM Dataset: Multi-Year Datasets for Longit...,Recent research has demonstrated the capabilit...,https://nips.cc/Conferences/2022/Schedule?show...
2,53451,Dynamics of SGD with Stochastic Polyak Stepsiz...,"Recently Loizou et al. (2021), proposed and an...",https://nips.cc/Conferences/2022/Schedule?show...
3,53636,Non-Stationary Bandits under Recharging Payoff...,The stochastic multi-armed bandit setting has ...,https://nips.cc/Conferences/2022/Schedule?show...
4,53782,Preservation of the Global Knowledge by Not-Tr...,"In federated learning, a strong global model i...",https://nips.cc/Conferences/2022/Schedule?show...


In [2]:
import torch
import config
from annoy import AnnoyIndex
from sentence_transformers import SentenceTransformer
import plotly.express as px

topk = 5
embeddings_dict = torch.load(config.EMBEDDINGS_PATH)
embed_size = embeddings_dict["embed_size"]

model = AnnoyIndex(embed_size, 'angular')
model.load(config.ANN_PATH) # super fast, will just mmap the file

/home/miniconda3/envs/pytorch_p39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
query = "cnn"
sbert_model = SentenceTransformer(config.MODEL_STR,device="cpu")


In [5]:
def load_data(data_path):
    json_data = json.load(open(data_path,"r"))
    paper_ids = []
    paper_texts = []
    for k,v in json_data.items():
        paper_ids.append(k)
        paper_texts.append( v["title"].strip()+". "+v["abstract"] )
    return paper_ids,paper_texts

In [7]:
_,paper_texts = load_data(config.DATA_PATH)

In [8]:
sbert_embeds = sbert_model.encode(paper_texts,batch_size=64*2,
                        show_progress_bar=True,
                        device="cuda" if torch.cuda.is_available() else "cpu"
        )

Batches: 100%|██████████| 26/26 [00:07<00:00,  3.34it/s]


In [ ]:
out = model.get_nns_by_vector(sbert_embeds, topk, search_k=-1, include_distances=True)

In [13]:
out

([2464, 2477, 2361, 1339, 2276],
 [1.046443223953247,
  1.0510444641113281,
  1.0534943342208862,
  1.0620694160461426,
  1.0669279098510742])